In [1]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon

import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster

In [2]:
def embed_map(m, file_name):
    """
    Generates an HTML file from a map object and displays it.
    """
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [3]:
%%bash
cp -r /Users/gurupratap.matharu/Downloads/geospatial-learn-course-data/NYPD_Motor_Vehicle_Collisions/ ../datasets/geospatial/

In [4]:
collisions = gpd.read_file("../datasets/geospatial/NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp")
collisions.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,NUMBER OF,NUMBER O_1,NUMBER O_2,NUMBER O_3,NUMBER O_4,NUMBER O_5,NUMBER O_6,NUMBER O_7,CONTRIBUTI,CONTRIBU_1,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,1.0,0.0,0,0,0,0,1,0,Driver Inattention/Distraction,Unspecified,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,1.0,0.0,0,0,0,0,1,0,Following Too Closely,Unspecified,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178)
3,07/30/2019,0:35,MANHATTAN,10036,40.756744,-73.984590,"(40.756744, -73.98459)",None,None,155 WEST 44 STREET,1.0,0.0,0,0,0,0,1,0,Oversized Vehicle,Unspecified,None,None,None,4179544,Box Truck,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (988519.261 214979.320)
4,07/30/2019,10:00,BROOKLYN,11223,40.600090,-73.965910,"(40.60009, -73.96591)",AVENUE T,OCEAN PARKWAY,None,1.0,0.0,0,0,1,0,0,0,Traffic Control Disregarded,Unspecified,None,None,None,4180660,Station Wagon/Sport Utility Vehicle,Bike,None,None,None,POINT (993716.669 157907.212)


Let's create an interactive map to visualize the collisions

In [5]:
m_1 = folium.Map(location=[40.7, -74], zoom_start=11)

# add heat map for collisions
HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_1)

embed_map(m_1, "NY_collisions.html")

In [6]:
%%bash 

cp -r /Users/gurupratap.matharu/Downloads/geospatial-learn-course-data/nyu_2451_34494/ ../datasets/geospatial/

Let's see the hospitals in this area

In [7]:
hospitals = gpd.read_file('../datasets/geospatial/nyu_2451_34494/nyu_2451_34494.shp')
hospitals.head()

,id,name,address,zip,factype,facname,capacity,capname,bcode,xcoord,ycoord,latitude,longitude,geometry
0,317000001H1178,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...,1650 Grand Concourse,10457,3102,Hospital,415,Beds,36005,1008872.0,246596.0,40.843490,-73.911010,POINT (1008872.000 246596.000)
1,317000001H1164,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 Fulton Ave,10456,3102,Hospital,164,Beds,36005,1011044.0,242204.0,40.831429,-73.903178,POINT (1011044.000 242204.000)
2,317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,10461,3102,Hospital,225,Beds,36005,1027505.0,248287.0,40.848060,-73.843656,POINT (1027505.000 248287.000)
3,317000002H1165,JACOBI MEDICAL CENTER,1400 Pelham Pkwy,10461,3102,Hospital,457,Beds,36005,1027042.0,251065.0,40.855687,-73.845311,POINT (1027042.000 251065.000)
4,317000008H1172,LINCOLN MEDICAL & MENTAL HEALTH CENTER,234 E 149 St,10451,3102,Hospital,362,Beds,36005,1005154.0,236853.0,40.816758,-73.924478,POINT (1005154.000 236853.000)


In [8]:
m_2 = folium.Map(location=[40.7, -74], zoom_start=11)

# add hospitals as markers
for idx, row in hospitals.iterrows():
    Marker(location=[row['latitude'], row['longitude']], popup=row['name']).add_to(m_2)

    
embed_map(m_2, "hospitals.html")

Find collisions that occurred more than 10km away from any hospital

In [9]:
# first see the coordinate reference system CRS
print(collisions.crs)
print(hospitals.crs)

{'init': 'epsg:2263'}
{'init': 'epsg:2263'}


epsg:2263 has a default unit of meters

In [10]:
# create a ten mile buffer
ten_mile_buffer = hospitals.geometry.buffer(10000)
ten_mile_buffer.head()

0    POLYGON ((1018872.000 246596.000, 1018823.847 ...
1    POLYGON ((1021044.000 242204.000, 1020995.847 ...
2    POLYGON ((1037505.000 248287.000, 1037456.847 ...
3    POLYGON ((1037042.000 251065.000, 1036993.847 ...
4    POLYGON ((1015154.000 236853.000, 1015105.847 ...
dtype: geometry

In [11]:
my_union = ten_mile_buffer.geometry.unary_union
type(my_union)

shapely.geometry.multipolygon.MultiPolygon

We use the multipolygon for efficiency instead of iterating through each polygon.

In [13]:
outside_range = collisions.loc[~collisions['geometry'].apply(lambda x: my_union.contains(x))]

In [14]:
outside_range.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,NUMBER OF,NUMBER O_1,NUMBER O_2,NUMBER O_3,NUMBER O_4,NUMBER O_5,NUMBER O_6,NUMBER O_7,CONTRIBUTI,CONTRIBU_1,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,1.0,0.0,0,0,0,0,1,0,Driver Inattention/Distraction,Unspecified,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
5,07/30/2019,10:50,QUEENS,11423,40.721060,-73.759450,"(40.72106, -73.75945)",FRANCIS LEWIS BOULEVARD,HILLSIDE AVENUE,None,2.0,0.0,0,0,0,0,2,0,Following Too Closely,Unspecified,None,None,None,4179812,Sedan,Box Truck,None,None,None,POINT (1050928.749 202069.687)
6,07/30/2019,10:55,QUEENS,11434,40.676228,-73.761120,"(40.676228, -73.76112)",CRANDALL AVENUE,CHENEY STREET,None,1.0,0.0,0,0,0,0,1,0,Unspecified,Unspecified,None,None,None,4180464,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1050510.380 185734.852)
15,07/30/2019,13:05,None,None,40.588413,-74.166725,"(40.588413, -74.166725)",None,None,26 RICHMOND HILL ROAD,1.0,0.0,0,0,0,0,1,0,Passing or Lane Usage Improper,Unspecified,None,None,None,4180091,Station Wagon/Sport Utility Vehicle,Box Truck,None,None,None,POINT (937943.004 153695.210)


So these were the collisions that occured ten kms away from any hospital

In [17]:
percentage = round(100 * len(outside_range) / len(collisions), 2)
print("Percentage of collisions more than 10km away from the closest hospital: {}%".format(percentage))

Percentage of collisions more than 10km away from the closest hospital: 15.12%


Build a Recommender
 - Given the location of any collision we should be able to provide the name of the nearest hospital

In [18]:
def best_hospital(collision_location):
    """
    Returns the name of the hospital closest the the collision location.
    """
    distances = hospitals.geometry.distance(collision_location)
    closest_hospital = hospitals.loc[distances.idxmin()]
    return closest_hospital['name']

In [19]:
print(best_hospital(outside_range.geometry.iloc[0]))

CALVARY HOSPITAL INC


Which hospital is in most demand?

In [20]:
recommended_hospitals = outside_range['geometry'].apply(lambda x: best_hospital(x))
most_visited_hospital = recommended_hospitals.value_counts().idxmax()
print(most_visited_hospital)

JAMAICA HOSPITAL MEDICAL CENTER


Where to construct new hospitals?

In [21]:
# create base map
m_6 = folium.Map(location=[40.7, -74], zoom_start=11)

# add ten mile coverage radius
folium.GeoJson(ten_mile_buffer.geometry.to_crs(epsg=4326)).add_to(m_6)

# add collisions as a heatmap
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(m_6)

# make lat long popup
folium.LatLngPopup().add_to(m_6)

# show map
embed_map(m_6, 'coverage_vs_collisions.html')

/Users/gurupratap.matharu/anaconda3/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/gurupratap.matharu/anaconda3/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


- So we see some high heated zones which are outside the coverage radius of any hospital
- There are good candidates for new hospitals

In [22]:
# proposed location of hospital 1
lat_1 = 40.69
long_1 = -73.75

# proposed location of hospital 2
lat_2 = 40.68
long_2 = -73.86

In [23]:
new_df = pd.DataFrame({'Latitude': [lat_1, lat_2],
                      'Longitude': [long_1, long_2]})
new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
new_gdf.crs = {'init': 'epsg:4326'}
new_gdf = new_gdf.to_crs(epsg=2263)

/Users/gurupratap.matharu/anaconda3/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [24]:
new_ten_mile_buffer = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)
new_my_union = new_ten_mile_buffer.unary_union

In [25]:
new_outside_range = outside_range.loc[~outside_range['geometry'].apply(lambda x: new_my_union.contains(x))]
new_percentage = round(100 *len(new_outside_range) / len(collisions), 2)
print("NEW percentage of collisions more than 10 km away from the closest hospital: {}%".format(new_percentage))

NEW percentage of collisions more than 10 km away from the closest hospital: 8.85%


Great! We are able to propose location for 2 new hospitals and reduce the collisions beyond 10 km reach to less than 10%

Let's visualize with a map one last time

In [29]:
m = folium.Map(location=[40.7, -74], zoom_start=11)
folium.GeoJson(ten_mile_buffer.geometry.to_crs(epsg=4326)).add_to(m)
folium.GeoJson(new_ten_mile_buffer.geometry.to_crs(epsg=4326)).add_to(m)

for idx, row in new_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m)

HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']]).add_to(m)

folium.LatLngPopup().add_to(m)

embed_map(m, 'final_result_hospital_vs_collisions.html')

/Users/gurupratap.matharu/anaconda3/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/gurupratap.matharu/anaconda3/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
/Users/gurupratap.matharu/anaconda3/lib/python3.7/site-packages/pyproj/crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))
